<a href="https://colab.research.google.com/github/ripunjaysingh88/ML_Models_from_Scratch/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from google.colab import drive
drive.mount('/content/grive')

Mounted at /content/grive


In [35]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
iris = load_iris()

In [4]:
original_X = iris.data
original_X.shape

(150, 4)

In [5]:
original_X.dtype

dtype('float64')

In [6]:
y = iris.target
y.shape

(150,)

In [7]:
y.dtype

dtype('int64')

In [8]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [9]:
original_X[1]

array([4.9, 3. , 1.4, 0.2])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(original_X, y, test_size = 0.20, random_state=42)

In [11]:
X_train.shape , X_test.shape

((120, 4), (30, 4))

In [12]:
y_train

array([0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2,
       1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2,
       1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1,
       0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2,
       1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2,
       1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

In [13]:
#gini_impurity
arr = (np.bincount(y_train)/len(y_train))**2
print(arr)
print()
1-np.sum(arr)

[0.11111111 0.11673611 0.105625  ]



0.6665277777777778

In [28]:
class Node:
  #internal nodes will have 'best split feature' on which decision are made. 'best threshold' is used in separating the data- left and right.
  #left and right stores address of the child node(tree).
  #When we are at leaf node then we need to store the most common classs label which we store in 'value'.
  def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  

class DecisionTree:
  def __init__(self, min_samples_split, max_depth):
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.root = None # need to know the root to traverse the tree latter for predicting samples.
  
  def most_common_label(self,y):
    return np.argmax(np.bincount(y))
  
  #lower the better
  def gini_impurity(self, y):
    arr = (np.bincount(y)/len(y))**2
    gini = 1 - np.sum(arr)
    return gini
  
  def split(self, X_column, threshold):
    # it stores 'list' not ndarray, we use this list to access numpy array.
    left_idx =  [index for index, value in enumerate(X_column) if value <= threshold]
    right_idx = [index for index, value in enumerate(X_column) if value > threshold] 
    return left_idx , right_idx
  
  def information_gain(self, X_column, y, threshold):
    #gini impurity of parent node
    gini_parent = self.gini_impurity(y)
    
    #split our data based on current feature(X_column) and particular 'threshold' value to calculate gini impurity of child nodes.
    left_index , right_index = self.split(X_column, threshold)
    if len(left_index) == 0 or len(right_index)==0 :
      return 0

    left_y = y[left_index]
    right_y = y[right_index]

    #gini impurity of child nodes
    gini_left_child = self.gini_impurity(left_y)
    gini_right_child = self.gini_impurity(right_y)

    #information gain 
    n_left = len(left_y)/len(y)
    n_right = len(right_y)/len(y)
    info_gain = gini_parent - ((n_left * gini_left_child) + (n_right * gini_right_child))
    return info_gain

  
  def best_criteria(self, X, y, feature_index):
    best_gain = -1 #gain range -> [0,1]
    best_feat , best_thresh = None, None

    for feat_index in feature_index:
      X_column = X[:, feat_index] #selecting particular column with all the samples.
      threshold_list = np.unique(X_column) # only unique values of column are selected.
      for threshold in threshold_list:
        #calculate info_gain on a particular feature with the particular threshold.
        #to calculate info_gain we need to calculate gini_impurity so 'y' passed.
        info_gain = self.information_gain(X_column, y, threshold) 
        if info_gain > best_gain:
          best_gain = info_gain
          best_feat= feat_index
          best_thresh = threshold
    return best_feat, best_thresh  


  def build_tree(self, X, y, depth = 0):
    n_samples , n_features = X.shape
    n_labels = len(np.unique(y))  #number of  classes e.g. binary classification or muticlass-classfication

    #stopping  criteria, if any condition satisfies then we reached leaf node.
    #depth starts from 0(so equality is used). If depth < max_depth, keep building tree else stop it and return leaf node.
    #n_labels, if a node has all the samples of same class then stop building tree as it's pure node. 
    if (depth>=self.max_depth or n_samples < self.min_samples_split or n_labels == 1):
      leaf_value = self.most_common_label(y)
      return Node(value = leaf_value)

    #'to select feature randomly' but it we will use all the features.Just oder in which it will select feature will be random.
    #first parameter is range (0 to n_features)
    #second parameter is number of random integers we want within that range.
    #np.random.choice(5, 5, replace = False) -> array([0, 1, 4, 3, 2]) return 5 number in the range of 0-5 randomly.
    #replace flase as we don't want the same feature to be checked again inoder ro find best_feature.
    feature_index = np.random.choice(n_features, n_features, replace=False)

    #it returns best feature(say X2(index will be 2 corrosponding to X2) out of {X0, X1, X2, ....,Xn})using which we split the data, 
    #which gives best gain in comparision to other features.
    #X2 is be a feature which we make it as a 'decision node' to split our data.
    #best threshold on which X2(best_feature) gave a maximum gain.
    best_feature, best_threshold = self.best_criteria(X, y, feature_index)
    
    #next to build our left and right tree using these best feature and best threshold.
    #get left_index and right_index to send new 'X' to build child trees.
    X_column = X[: , best_feature]
    left_index, right_index = self.split(X_column, best_threshold)
    #only particular rows with all columns
    X_left , y_left = X[left_index , : ] , y[left_index]
    X_right , y_right = X[right_index , :], y[right_index]
    
    #return value is stored bec we need to store the child trees.
    left = self.build_tree(X_left, y_left, depth+1) 
    right = self.build_tree(X_right, y_right, depth+1)
    #return internal node i.e the decision node.
    return Node(best_feature, best_threshold, left, right, None) 

    
  def fit(self, X, y):
    #build the tree and store root node for traversing tree to predict.
    self.root = self.build_tree(X, y)
    #return self.root

  def predict(self, X):
    #traverse the tree to predict
    #predicting one sample at a time. 'x' is one sample.
    return np.array([self.traverse_tree(x, self.root) for x in X]) 
  
  def traverse_tree(self, x, root):
    #if leave node reached then return value
    if root.value is not None:
      return root.value
    
    #say if root.feature is 2 then we will select second feature from the 'x'.
    feature_index = root.feature
    feature_value = x[feature_index] #say x[2] is 11.
    #call left sub-tree
    if feature_value <= root.threshold:
      return self.traverse_tree(x, root.left)
    #call right-subtree
    else:
      return self.traverse_tree(x, root.right)


In [49]:
#depth of tree should be 10 , minimum number of sample to split further should be 10 .
decision_tree_clf = DecisionTree(10, 10) 
decision_tree_clf.fit(X_train, y_train)
y_pred = decision_tree_clf.predict(X_test)

In [50]:
print("Accuracy : {}".format(accuracy_score(y_test, y_pred)))

Accuracy : 1.0


In [51]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [52]:
#sk-learn decision-tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)
y_pred_sklearn = clf.predict(X_test)

In [53]:
print("Accuracy sklearn : {}".format(accuracy_score(y_test, y_pred_sklearn)))

Accuracy sklearn : 1.0


In [54]:
print(classification_report(y_test,y_pred_sklearn))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [24]:
#root = decision_tree_clf.fit(X_train, y_train)

In [25]:
#root.threshold

1.9

In [26]:
#root.feature

2

In [27]:
#root.left